# Intro to Julia

In this lecture we will learn about one of my favorite programming languages: Julia

Julia is a new language:

- first public release (v0.1) in 2013
- First API stable release (v1.0) in 2018
- I've been actively using and developing Julia libraries since 2014 (v0.2)

Without Julia the code for my PhD would still be running and I wouldn't be Dr. Lyon...

## Why Julia?

Julia is a generic programming language: 

- Can be used for any task
- Is used for a variety of applications such as data analytics, web programming, microservices

However, it is also built with numerical/technical computing in mind

Has many features that help the scientific programmer:

- Built in nd-array types
- Extensive standard library (shipped with Julia itself): `Dates`, `Test`, `SparseArrays`, `Distributed`, `LinearAlgebra`, `Random`, `Markdown`, and many more!
- Built in tools for multi threaded concurrency and distributed parallel computation
- Julia code is *very fast*


In [1]:
1+1

2

In [2]:
function foo()
    out = BigInt(0)
    for i in 1:10_000_000_000    # range(1, 10_000_000_000)
        out += i
    end
    return out
end

foo (generic function with 1 method)

In [ ]:
@time foo()

In [3]:
@code_lowered foo()

CodeInfo(
1 ─       out = Main.BigInt(0)
│   %2  = 1:10000000000
│         @_2 = Base.iterate(%2)
│   %4  = @_2 === nothing
│   %5  = Base.not_int(%4)
└──       goto #4 if not %5
2 ┄ %7  = @_2
│         i = Core.getfield(%7, 1)
│   %9  = Core.getfield(%7, 2)
│         out = out + i
│         @_2 = Base.iterate(%2, %9)
│   %12 = @_2 === nothing
│   %13 = Base.not_int(%12)
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return out
)

In [4]:
@code_typed foo()

CodeInfo(
1 ── %1  = Base.GMP.MPZ.set_si::typeof(Base.GMP.MPZ.set_si)
│    %2  = invoke %1(0::Int64)::BigInt
└───       goto #16 if not true
2 ┄─ %4  = φ (#1 => 1, #15 => %35)::Int64
│    %5  = φ (#1 => 1, #15 => %36)::Int64
│    %6  = φ (#1 => %2, #15 => %28)::BigInt
│    %7  = Base.slt_int(%4, 0)::Bool
└───       goto #4 if not %7
3 ── %9  = Base.bitcast(UInt64, %4)::UInt64
│    %10 = Base.neg_int(%9)::UInt64
│    %11 = Base.GMP.MPZ.sub_ui::typeof(Base.GMP.MPZ.sub_ui)
│    %12 = invoke %11(%6::BigInt, %10::UInt64)::BigInt
└───       goto #11
4 ── %14 = Core.lshr_int(%4, 63)::Int64
│    %15 = Core.trunc_int(Core.UInt8, %14)::UInt8
│    %16 = Core.eq_int(%15, 0x01)::Bool
└───       goto #6 if not %16
5 ──       invoke Core.throw_inexacterror(:check_top_bit::Symbol, UInt64::Type{UInt64}, %4::Int64)::Union{}
└───       unreachable
6 ──       goto #7
7 ── %21 = Core.bitcast(Core.UInt64, %4)::UInt64
└───       goto #8
8 ──       goto #9
9 ──       goto #10
10 ─ %25 = Base.GMP.MPZ.add_ui::t

In [5]:
@code_llvm foo()

;  @ In[2]:1 within `foo`
define nonnull {}* @julia_foo_1898() #0 {
top:
  %gcframe9 = alloca [3 x {}*], align 16
  %gcframe9.sub = getelementptr inbounds [3 x {}*], [3 x {}*]* %gcframe9, i64 0, i64 0
  %0 = bitcast [3 x {}*]* %gcframe9 to i8*
  call void @llvm.memset.p0i8.i32(i8* nonnull align 16 dereferenceable(24) %0, i8 0, i32 24, i1 false)
  %thread_ptr = call i8* asm "movq %fs:0, $0", "=r"() #4
  %ppgcstack_i8 = getelementptr i8, i8* %thread_ptr, i64 -8
  %ppgcstack = bitcast i8* %ppgcstack_i8 to {}****
  %pgcstack = load {}***, {}**** %ppgcstack, align 8
;  @ In[2]:2 within `foo`
; ┌ @ gmp.jl:291 within `BigInt`
   %1 = bitcast [3 x {}*]* %gcframe9 to i64*
   store i64 4, i64* %1, align 16
   %2 = getelementptr inbounds [3 x {}*], [3 x {}*]* %gcframe9, i64 0, i64 1
   %3 = bitcast {}** %2 to {}***
   %4 = load {}**, {}*** %pgcstack, align 8
   store {}** %4, {}*** %3, align 8
   %5 = bitcast {}*** %pgcstack to {}***
   store {}** %gcframe9.sub, {}*** %5, align 8
   %6 = call non

In [6]:
@code_native foo()

	.text
; ┌ @ In[2]:1 within `foo`
	pushq	%r15
	pushq	%r14
	pushq	%r12
	pushq	%rbx
	subq	$24, %rsp
	vxorps	%xmm0, %xmm0, %xmm0
	movq	$0, 16(%rsp)
	movq	%fs:0, %rax
	movq	%rsp, %rcx
; │ @ In[2]:2 within `foo`
; │┌ @ gmp.jl:291 within `BigInt`
	xorl	%edi, %edi
	vmovaps	%xmm0, (%rsp)
	movq	-8(%rax), %r14
	movq	$4, (%rsp)
	movq	(%r14), %rax
	movq	%rax, 8(%rsp)
	movabsq	$set_si, %rax
	movq	%rcx, (%r14)
	callq	*%rax
	movl	$1, %ebx
	movabsq	$add_ui, %r15
	movabsq	$10000000001, %r12              # imm = 0x2540BE401
	nopw	(%rax,%rax)
; │└
; │ @ In[2]:4 within `foo`
; │┌ @ gmp.jl:534 within `+` @ gmp.jl:528
L112:
	movq	%rax, %rdi
	movq	%rbx, %rsi
	movq	%rax, 16(%rsp)
	callq	*%r15
; │└
; │┌ @ range.jl:837 within `iterate`
	incq	%rbx
; ││┌ @ promotion.jl:468 within `==`
	cmpq	%r12, %rbx
; │└└
	jne	L112
; │ @ In[2] within `foo`
	movq	8(%rsp), %rcx
	movq	%rcx, (%r14)
; │ @ In[2]:6 within `foo`
	addq	$24, %rsp
	popq	%rbx
	popq	%r12
	popq	%r14
	popq	%r15
	retq
	nopw	(%rax,%rax)
; └


In [ ]:
function bar(n)
    return rand(n) .+ 10
end

```python
def foo():
    out = 0
    for i in range(10):
        out += 1
    return out
```           

1. set out = 0
2. set i = 0
3. set out = out + 1 = 1
4. set i = 1
5. set out = out + 1 = 2
4. set i = 2
5. set out = out + 1 = 3
4. set i = 3
5. set out = out + 1 = 4
. .........
1. return 10

### Julia's Technology

Julia's technology stack makes its unique value propositions possible

Julia is a just in time (JIT) compiled language using the LLVM toolchain (powers Rust, modern C++ compilers, etc... **very** efficient)

The JITted nature of Julia means:

- All code is compiled to machine code before execution
- This happens as needed, so it feels as dynamic as interpreted languages like Python
- The machine code can be optimized end to end and execute at C-like speeds


### Julia is written in Julia

Julia has a relatively small core written in C

The vast majority of Julia itself, its standard libraries, and third party packages are written in Julia

This means a few things:

- Julia users can be library developers/contributors (open source FTW)
- Julia user code has same standing as built in code (not true for languages like Python/R)
- Julia doesn't often have "two language problem": users write most code in one language and are forced to use a second language to make it fast (e.g. Python + C)

### Other Language features

Julia was originally created to make numerical computing better

*warning, opinion coming!*

As such, Julia's syntax is concice, expressive, and beautiful for doing numerical work

Julia has advanced features like metaprogramming (code that writes code!), parametric types, symmetric coroutines, etc...

The result is a system that allows an ecosystem of *composable* and *interoperable* components (a User can use features of one package with another package without either package knowing the other exists)

## Installation

To install Julia on your machine...

- Go to https://julialang.org/downloads/
- Download the latest released version of Julia for your operating system and follow system-specific installation instructions
- Install jupyter support (optional): Launch julia then at prompt type: `using Pkg; Pkg.add("IJulia")`